In [15]:
import os
os.chdir("/Users/boxiang/Documents/!-work/Baidu/projects/sim_anneal/scripts/")
import sys

from utils.utils import read_fasta, cfg_file, load_config, CAI
import numpy as np
from tqdm import tqdm
import pickle as pkl
import subprocess

from collections import defaultdict
import pandas as pd
from plotnine import ggplot, geom_point, geom_line, aes, theme_bw, \
    scale_color_manual, scale_shape_manual, theme, element_blank

In [2]:
from cai_random.optimize import CAIOptimizer, Plotter

In [3]:
# initialize:
cfg = load_config(cfg_file)
seqs = read_fasta(cfg.DATA.RAW.SPIKE)
mfe_seq = seqs["lambda_0"]
cai_seq = seqs["lambda_inf"]
cai_calc = CAI(cfg.DATA.RAW.CODON)

In [4]:
# mutate sequence and calculate MFE & CAI:
optimizer = CAIOptimizer(mfe_seq, cai_seq)
results = dict()
for proportion in tqdm(np.arange(0, 1.001, 0.02)):
    for seed in np.arange(1):
        mut_seq = optimizer.mutate(proportion, seed=seed)
        mfe = optimizer.get_mfe(seq=mut_seq, folding_cmd=cfg.BIN.RNAFOLD)
        cai = cai_calc.get_cai(mut_seq)
        results[(proportion, seed)] = [mut_seq, mfe, cai]
        
pkl_file = os.path.join(
    cfg.DATA.PROCESSED.CAI_RANDOM, "mutate_results.pkl")
with open(pkl_file, "wb") as f:
    pkl.dump(results, f)

100%|██████████| 51/51 [13:40<00:00, 16.10s/it]


In [5]:
pkl_file = os.path.join(
    cfg.DATA.PROCESSED.CAI_RANDOM, "mutate_results.pkl")
with open(pkl_file, "rb") as f:
    results = pkl.load(f)

In [16]:
# make MFE-CAI plot:
ref_points = pd.read_csv(cfg.DATA.RAW.REF_P)
cai_random = [(k[0], k[1], v[1], v[2]) for k, v in results.items()]
cai_random = pd.DataFrame(
    cai_random, columns=["proportion", "seed", "MFE", "CAI"])

plotter = Plotter(ref_points)
plotter.add_points(cai_random, name="Random Optimization", color="red", shape="^")
p = plotter.plot()
p.save(os.path.join(cfg.DATA.PROCESSED.CAI_RANDOM, "comparison.pdf"))

/Users/boxiang/miniconda3/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/boxiang/miniconda3/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: /Users/boxiang/Documents/!-work/Baidu/projects/sim_anneal/data/processed/cai_random/comparison.pdf
